<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#4.1.-NumPy-indexing" data-toc-modified-id="4.1.-NumPy-indexing-1">4.1. NumPy indexing</a></span></li><li><span><a href="#4.2.-Color-images" data-toc-modified-id="4.2.-Color-images-2">4.2. Color images</a></span></li><li><span><a href="#4.3.-Coordinate-conventions" data-toc-modified-id="4.3.-Coordinate-conventions-3">4.3. Coordinate conventions</a></span></li><li><span><a href="#4.4.-Notes-on-the-order-of-array-dimensions" data-toc-modified-id="4.4.-Notes-on-the-order-of-array-dimensions-4">4.4. Notes on the order of array dimensions</a></span></li><li><span><a href="#4.5.-A-note-on-the-time-dimension" data-toc-modified-id="4.5.-A-note-on-the-time-dimension-5">4.5. A note on the time dimension</a></span></li></ul></div>

# 4. A crash course on NumPy for images

`scikit-image` 中的图像由 NumPy ndarrays 表示。因此，可以使用标准 NumPy 方法来操作 arrays 来实现许多常见操作：

In [3]:
from skimage import data
camera = data.camera()
type(camera)

numpy.ndarray

In [ ]:
检索图像的几何形状和像素数：

In [2]:
camera.shape

(512, 512)

In [3]:
camera.size

262144

检索有关图像强度值的统计信息：

In [4]:
camera.min(), camera.max()

(0, 255)

In [5]:
camera.mean()

129.06072616577148

表示图像的 NumPy 数组可以是不同的整数或浮点数值类型。有关这些类型以及 `scikit-image` 如何处理它们的更多信息，请参阅 [Image data types and what they mean](https://scikit-image.org/docs/stable/user_guide/data_types.html#data-types)。

## 4.1. NumPy indexing

NumPy 索引既可用于查看像素值，也可用于修改它们：

In [6]:
# Get the value of the pixel at the 10th row and 20th column
camera[10, 20]

200

In [7]:
# Set to black the pixel at the 3rd row and 10th column
camera[3, 10] = 0

当心！在 NumPy 索引中，第一个维度 (`camera.shape[0]`) 对应于行，而第二个维度 (`camera.shape[1]`) 对应于列，原点 (`camera[0, 0]`) 位于左上角角落。这与矩阵/线性代数表示法匹配，但与笛卡尔 (x, y) 坐标相反。有关更多详细信息，请参阅下面的 [Coordinate conventions](https://scikit-image.org/docs/stable/user_guide/numpy_images.html#coordinate-conventions)。

除了单个像素之外，还可以使用 NumPy 的不同索引功能来访问/修改整组像素的值。

切片：

In [8]:
# Set the first ten lines to "black" (0)
camera[:10] = 0

掩码（使用布尔值掩码进行索引）：

In [9]:
mask = camera < 87
# Set to "white" (255) the pixels where mask is True
camera[mask] = 255

花式索引（使用索引集进行索引）：

In [4]:
import numpy as np
inds_r = np.arange(len(camera))
inds_c = 4 * inds_r % len(camera)
camera[inds_r, inds_c] = 0

当您需要选择一组像素来执行操作时，掩码（Masks）非常有用。掩码可以是与图像形状相同的任何布尔数组（或可广播到图像形状的形状）。这可用于定义感兴趣的区域，例如磁盘形状：

In [12]:
nrows, ncols = camera.shape
row, col = np.ogrid[:nrows, :ncols]
cnt_row, cnt_col = nrows / 2, ncols / 2
outer_disk_mask = ((row - cnt_row)**2 + (col - cnt_col)**2 >
                   (nrows / 2)**2)
camera[outer_disk_mask] = 0

![camera](https://scikit-image.org/docs/stable/_images/sphx_glr_plot_camera_numpy_001.png)

NumPy 的布尔运算可用于定义更复杂的掩码：

In [14]:
lower_half = row > cnt_row
lower_half_disk = np.logical_and(lower_half, outer_disk_mask)
camera = data.camera()
camera[lower_half_disk] = 0

## 4.2. Color images

对于彩色图像，上述所有内容仍然适用。彩色图像是一个 NumPy array，带有额外的通道尾随维度：

In [3]:
cat = data.chelsea()
type(cat)

numpy.ndarray

In [4]:
>>> cat.shape
(300, 451, 3)

(300, 451, 3)

这表明 `cat` 是一个 300 x 451 像素的图像，具有三个通道（红色、绿色和蓝色）。和以前一样，我们可以获取和设置像素值：

In [5]:
cat[10, 20]

array([151, 129, 115], dtype=uint8)

In [6]:
# Set the pixel at (50th row, 60th column) to "black"
cat[50, 60] = 0
# set the pixel at (50th row, 61st column) to "green"
cat[50, 61] = [0, 255, 0]  # [red, green, blue]

我们还可以对 2D 多通道图像使用 2D 布尔蒙版，就像我们对上面的灰度图像所做的那样：

![](https://scikit-image.org/docs/stable/_images/numpy_images-1.png)

`skimage.data` 中包含的示例彩色图像具有沿最后一个轴存储的通道，但其他软件可能遵循不同的约定。支持彩色图像的 scikit-image 库函数有一个 `channel_axis` 参数，可用于指定数组的哪个轴对应于通道。

## 4.3. Coordinate conventions

由于 `scikit-image` 使用 NumPy arrays 表示图像，因此坐标约定必须匹配。二维 (2D) 灰度图像（例如上面的 camera）按行和列（缩写为 `(row, col)` 或 `(r, c)`）进行索引，最低元素 `(0, 0)` 在顶部-左角。在库的各个部分，您还会看到 `rr` 和 `cc` 指的是行和列坐标列表。我们将此约定与 `(x, y)` 区分开来，后者通常表示标准笛卡尔坐标，其中 `x` 是水平坐标，`y` 是垂直坐标，原点位于左下角（例如，Matplotlib 轴就使用此约定） 。

在多通道图像的情况下，任何维度（array axis）都可以用于颜色通道，并用 `channel` 或 `ch` 表示。在 scikit-image 0.19 之前，此通道维度始终是最后一个，但在当前版本中，通道维度可以通过 `channel_axis` 参数指定。需要多通道数据的函数默认为 `channel_axis=-1`。否则，函数默认为`channel_axis=None`，表示假设没有轴对应于通道。

最后，对于体积 (3D) 图像，例如视频、磁共振成像 (MRI) 扫描、共焦显微镜等，我们将主维度称为`plane`，缩写为 `pln` 或 `p`。

这些约定总结如下：

<center>Dimension name and order conventions in scikit-image</center>

|Image type|Coordinates|
|:-|:-|
|2D grayscale|(row, col)|
|2D multichannel (eg. RGB)|(row, col, ch)|
|3D grayscale|(pln, row, col)|
|3D multichannel|(pln, row, col, ch)|

请注意，`ch` 的位置由 `channel_axis` 参数控制。

`scikit-image` 中的许多函数可以直接操作 3D 图像：

In [ ]:
rng = np.random.default_rng()
im3d = rng.random((100, 1000, 1000))
from skimage.segmentation import watershed
from scipy import ndimage as ndi
seeds = ndi.label(im3d < 0.1)[0]
ws = watershed(im3d, seeds)

然而，在许多情况下，第三个空间维度的分辨率低于其他两个空间维度。一些 `scikit-image` 函数提供了 `spacing` 关键字参数来帮助处理此类数据：

In [ ]:
from skimage import segmentation
slics = segmentation.slic(im3d, spacing=[5, 1, 1], channel_axis=None)

其他时候，处理必须在平面上完成。当平面沿主维度堆叠时（与我们的约定一致），可以使用以下语法：

In [ ]:
from skimage import filters
edges = np.empty_like(im3d)
for pln, image in enumerate(im3d):
    # Iterate over the leading dimension
    edges[pln] = filters.sobel(image)

## 4.4. Notes on the order of array dimensions 

尽管轴的标签看起来可能是任意的，但它会对操作速度产生重大影响。这是因为现代处理器从不只从内存中检索一个项目，而是检索一整块相邻的项目（称为 prefetching 操作）。因此，即使操作次数相同，处理内存中彼此相邻的元素比处理分散的元素要快：

In [1]:
def in_order_multiply(arr, scalar):
    for plane in list(range(arr.shape[0])):
        arr[plane, :, :] *= scalar

def out_of_order_multiply(arr, scalar):
    for plane in list(range(arr.shape[2])):
        arr[:, :, plane] *= scalar

In [5]:
import time
rng = np.random.default_rng()
im3d = rng.random((100, 1024, 1024))
t0 = time.time(); x = in_order_multiply(im3d, 5); t1 = time.time()
print("%.2f seconds" % (t1 - t0))  

0.05 seconds


In [6]:
s0 = time.time(); x = out_of_order_multiply(im3d, 5); s1 = time.time()
print("%.2f seconds" % (s1 - s0)) 

1.35 seconds


In [7]:
print("Speedup: %.1fx" % ((s1 - s0) / (t1 - t0)))  

Speedup: 26.9x


当最后一个/最右边的维度变得更大时，加速会更加显着。在开发算法时值得考虑数据局部性。特别是，`scikit-image` 默认使用 C-连续数组。使用嵌套循环时，数组的最后一个/最右边的维度应该位于计算的最内层循环中。在上面的示例中，`*=` numpy 运算符迭代所有剩余维度。

## 4.5. A note on the time dimension

尽管 `scikit-image` 目前不提供专门处理时变 3D 数据的函数，但它与 NumPy 数组的兼容性使我们能够非常自然地处理形状 (t、pln、row、col、ch) 的 5D 数组：

In [ ]:
for timepoint in image5d:
    # Each timepoint is a 3D multichannel image
    do_something_with(timepoint)

我们可以对上表进行如下补充：

<center>Addendum to dimension names and orders in scikit-image</center>

|Image type|coordinates|
|:-|:-|
|2D color video|(t, row, col, ch)|
|3D color video|(t, pln, row, col, ch)|